In [32]:
import shapely.geometry
import scipy
from IPython.display import display
import glob
from datetime import datetime
import netCDF4 as nc
import netCDF4
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import glob
import os
from netCDF4 import date2num,num2date
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import xarray as xr
import geopandas as gpd
import numpy as np
import fiona


In [ ]:
def extract_nc (nc_path,mask_file): 
    mask = fiona.open(mask_file)
    with xr.open_dataset(nc_path) as file_nc:
        temp=file_nc
    aoi_lat = [int(mask.bounds[1])+np.array(temp.lat[0]-temp.lat[1]).item(), int(mask.bounds[3])+1]
    aoi_lon = [int(mask.bounds[0])+np.array(temp.lon[0]-temp.lon[1]).item(), int(mask.bounds[2])+1]
    for i in temp.dims:
        if i in["longitude","lon"]:
            lon=i
    lon_max = np.array(temp[lon]).max()
    if lon_max > 180:
        aoi_lon[0] = aoi_lon[0] + 180
        aoi_lon[1] = aoi_lon[1] + 180
        extract_nc= temp.sel(lat=slice(int(aoi_lat[0]),int(aoi_lat[1])),lon=slice(int(aoi_lon[0]),int(aoi_lon[1])))
    else:
        extract_nc= temp.sel(lat=slice(int(aoi_lat[0]),int(aoi_lat[1])),lon=slice(int(aoi_lon[0]),int(aoi_lon[1])))
    return extract_nc

In [4]:
def save_extract_nc(dataset,variable,gcm,ssp,variable_unit):
    # create and save the extracted nc file
    if type(dataset.indexes['time']).__name__ == "CFTimeIndex":      
      datetime_nc = dataset.indexes["time"].strftime("%m/%d/%Y, %H:%M:%S")
      datetime_nc = pd.to_datetime(datetime_nc).to_pydatetime()
      times1 = date2num(datetime_nc, "days since 1850-01-01 00:00:00")
    else:
      datetime_nc = dataset.indexes['time'].to_pydatetime()
      times1 = date2num(datetime_nc, "days since 1850-01-01 00:00:00")
    fn = "D:/CMIP6/Process/Jialingjiang/" +variable+"_day_"+gcm+"_"+ssp+"_r1i1p1f1_gn_"+str(datetime_nc[1].year)+"0101_"+ str(datetime_nc[-1].year)+".nc"
    ds = nc.Dataset(fn, 'w', format='NETCDF4')
    time = ds.createDimension('time', None)
    len_lat= len(dataset.lat)
    len_lon= len(dataset.lon)
    lat = ds.createDimension('lat', len_lat)
    lon = ds.createDimension('lon', len_lon)
    times = ds.createVariable('time', 'f4', ('time',))
    lats = ds.createVariable('lat', 'f4', ('lat',))
    lons = ds.createVariable('lon', 'f4', ('lon',))
    temp_variable = variable
    climate_variable = ds.createVariable(temp_variable, 'f4', ('time', 'lat', 'lon'))
    climate_variable.units = variable_unit
    times.units = "days since 1850-01-01 00:00:00"
    times[:] = times1
    lats[:] = np.array(dataset.lat[:])
    lons[:] = np.array(dataset.lon[:])   
    climate_variable[:]=np.array(dataset[variable][::])     
    ds.close()
    return len_lat,len_lon

In [8]:
list = glob.glob("F:/CMIP6/*/*/*/*.nc")
list_name = [i.split("\\")[-1].split("_")[2] for i in list]
GCM= sorted(set(list_name))
GCM
variable = ['pr','tasmax','tasmin']
ssp =['historical','ssp126','ssp245','ssp370','ssp585']
mask_file= "G:/Shared drives/ZW_workspace/Projects/Jialingjiang/Data/Boundary/Jialingjiang_GCS.shp"
for i in GCM:
    for j in variable:
        for k in ssp:
            file = glob.glob("F:/CMIP6/Process/Jialingjiang/" +j+"_day_"+i+"_"+k+"*.nc")
            print(i,j,k)
            path = r'F:/CMIP6/' + i +'/'+j+'/'+k
            list = glob.glob(path+"/*.nc")
            dataset = xr.merge([extract_nc(e,mask_file) for e in list], compat="no_conflicts")
            if variable == "pr":
             save_extract_nc(dataset,j,i,k,"kg m-2 s-1")
            else:
             save_extract_nc(dataset,j,i,k,"K") 


INM-CM4-8 tasmax ssp370


INM-CM4-8 tasmax ssp585


INM-CM4-8 tasmin historical


INM-CM4-8 tasmin ssp126


INM-CM4-8 tasmin ssp245


INM-CM4-8 tasmin ssp370


INM-CM4-8 tasmin ssp585


INM-CM5-0 pr historical


INM-CM5-0 pr ssp126


INM-CM5-0 pr ssp245


INM-CM5-0 pr ssp370


INM-CM5-0 pr ssp585


INM-CM5-0 tasmax historical


INM-CM5-0 tasmax ssp126


INM-CM5-0 tasmax ssp245


INM-CM5-0 tasmax ssp370


INM-CM5-0 tasmax ssp585


INM-CM5-0 tasmin historical


INM-CM5-0 tasmin ssp126


INM-CM5-0 tasmin ssp245


INM-CM5-0 tasmin ssp370


INM-CM5-0 tasmin ssp585


MPI-ESM1-2-HR pr historical









MPI-ESM1-2-HR pr ssp126


















MPI-ESM1-2-HR pr ssp245


















MPI-ESM1-2-HR pr ssp370


















MPI-ESM1-2-HR pr ssp585


















MPI-ESM1-2-HR tasmax historical









MPI-ESM1-2-HR tasmax ssp126


















MPI-ESM1-2-HR tasmax ssp245


















MPI-ESM1-2-HR tasmax ssp370


















MPI-ESM1-2-HR tasmax ssp585


















MP